In [1]:
import torch
import torch.utils.data as data

import numpy as np

In [2]:
from torch import nn, optim as opts
from plasma.training import Trainer, metrics, callbacks
from plasma.modules import commons

In [3]:
X = np.random.random((3, 28, 28))
y = np.random.randint(0, 10)

class Data(data.Dataset):
    
    def __len__(self):
        return 64 * 50
    
    def __getitem__(self, idx):
        return X, y

In [4]:
model = nn.Sequential(*[
    nn.Conv2d(3, 64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(128, 128, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(7, 7),
    commons.Reshape(-1),
    nn.Linear(256, 10),
    #nn.Softmax(dim=-1)
])

model.cuda(0)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=7, stride=7, padding=0, dilation=1, ceil_mode=False)
  (17): Reshape(shape=(-1,))
  (18

In [5]:
loss = nn.CrossEntropyLoss()

In [6]:
opt = opts.RMSprop(model.parameters())
#opt = opts.SGD(model.parameters(), lr=1.5, momentum=0.9, nesterov=True)
trainer = Trainer(model, opt, loss)

In [7]:
cbs = [
    #callbacks.LrFinder(min_lr=1e-6, max_lr=12, epochs=3)
    #callbacks.WarmRestart(1e-5, 1, factor=2),
    #callbacks.CLR(1e-5, 4),
    callbacks.TrainingScheduler(epochs=1)
]

In [8]:
trainer.fit(Data(), callbacks=cbs, batch_size=64)

epoch 1
